# Desafio Prático: Módulo 1: Fundamentos em Arquitetura de Dados e Soluções em Nuvem

# 0.0. Loading data

In [1]:
!pip install unidecode
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=c477ef31bfc55b54f712ac36f3ce30bb395695b050c4f9f005b76ef91490d74f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


## 0.1. Import libraries

In [2]:
import os
import re
import time
import requests
import subprocess

from functools             import reduce
from unidecode             import unidecode
from pyspark.sql           import SparkSession
from pyspark.sql.types     import StringType
from pyspark.sql.utils     import AnalysisException
from pyspark.sql.functions import col, regexp_replace, lower, translate, mean, max, min, col, count, avg, round, udf

from pyspark.sql           import functions as F

## 0.2 Import file

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
# from ftplib import FTP
# import zipfile
# import shutil

# # FTP server details
# ftp_url = "ftp.mtps.gov.br"
# ftp_path = "/pdet/microdados/RAIS/2020"
# output_folder = "/content/data/raw"  # Modify this to your desired local directory

# # Connect to the FTP server
# ftp = FTP(ftp_url)
# ftp.login()  # Login anonymously

# # Change to the desired directory on the FTP server
# ftp.cwd(ftp_path)

# # Create the output folder if it doesn't exist
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # List all files in the current directory on the FTP server
# file_list = ftp.nlst()

# # Get the list of files in the output folder
# files = os.listdir(output_folder)

# # Download and extract each file
# for filename in file_list:
#     local_filepath = os.path.join(output_folder, filename)

#     # Download the file
#     with open(local_filepath, "wb") as file:
#         ftp.retrbinary("RETR " + filename, file.write)
#         print(f"Downloaded: {filename}")

# # Disconnect from the FTP server
# ftp.quit()

# print("All files downloaded successfully.")

# # Path to the folder with .7z files
# source_folder = '/content/data/raw'

# # Get the list of files in the source folder
# files = os.listdir(source_folder)

# # Iterate over all files in the folder
# for file in files:
#     # Check if the file is a .7z file
#     if file.endswith('.7z'):
#         # Extract the file using 7-Zip (make sure you have 7-Zip installed)
#         command = f'7z x "{os.path.join(source_folder, file)}" -o"{source_folder}"'
#         os.system(command)
        
#         # Delete the .7z file
#         os.remove(os.path.join(source_folder, file))

# print("Extraction and deletion completed successfully.")

Downloaded: RAIS_ESTAB_PUB.7z
Downloaded: RAIS_VINC_PUB_CENTRO_OESTE.7z
Downloaded: RAIS_VINC_PUB_MG_ES_RJ.7z
Downloaded: RAIS_VINC_PUB_NORDESTE.7z
Downloaded: RAIS_VINC_PUB_NORTE.7z
Downloaded: RAIS_VINC_PUB_SP.7z
Downloaded: RAIS_VINC_PUB_SUL.7z
All files downloaded successfully.
Extraction and deletion completed successfully.


In [ ]:
# import shutil

# # Specify the source and destination paths
# source_path = '/content/data/raw'
# destination_path = '/content/drive/MyDrive/desafio_mba/rais/raw'

# # Copy the directory to your Google Drive
# shutil.copytree(source_path, destination_path)

'/content/drive/MyDrive/desafio_mba/rais/raw'

## 0.3 Load and change data

In [19]:
# Create Spark session
spark = SparkSession.builder.appName("ReadDataRAIS").getOrCreate()

rais = (
    spark
    .read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .option("delimiter", ";")
    .option("encoding", "latin1")
    .load("/content/drive/MyDrive/desafio_mba/rais/raw") 
) 

# Print the updated schema
rais.printSchema()

root
 |-- Bairros SP: string (nullable = true)
 |-- Bairros Fortaleza: string (nullable = true)
 |-- Bairros RJ: string (nullable = true)
 |-- Causa Afastamento 1: integer (nullable = true)
 |-- Causa Afastamento 2: integer (nullable = true)
 |-- Causa Afastamento 3: integer (nullable = true)
 |-- Motivo Desligamento: integer (nullable = true)
 |-- CBO Ocupação 2002: string (nullable = true)
 |-- CNAE 2.0 Classe: integer (nullable = true)
 |-- CNAE 95 Classe: integer (nullable = true)
 |-- Distritos SP: string (nullable = true)
 |-- Vínculo Ativo 31/12: integer (nullable = true)
 |-- Faixa Etária: integer (nullable = true)
 |-- Faixa Hora Contrat: integer (nullable = true)
 |-- Faixa Remun Dezem (SM): integer (nullable = true)
 |-- Faixa Remun Média (SM): integer (nullable = true)
 |-- Faixa Tempo Emprego: integer (nullable = true)
 |-- Escolaridade após 2005: double (nullable = true)
 |-- Qtd Hora Contr: double (nullable = true)
 |-- Idade: double (nullable = true)
 |-- Ind CEI Vincul

In [20]:
# Manually rename columns without accents
new_column_names = [unidecode(col_name) for col_name in rais.columns]
rais = rais.toDF(*new_column_names)

# rename columns without accents, lowercase, and replace spaces with underscores
new_column_names = [unidecode(col_name).lower().replace(" ", "_") for col_name in rais.columns]
rais = rais.toDF(*new_column_names)

# Add a new column "uf" by extracting the first two characters from the "municipio" column
rais = rais.withColumn("uf", col("municipio").cast("string").substr(1, 2).cast("int"))

# List of columns to be casted to double
double_columns = [
    "vl_rem_janeiro_sc",
    "vl_rem_fevereiro_sc",
    "vl_rem_marco_sc",
    "vl_rem_abril_sc",
    "vl_rem_maio_sc",
    "vl_rem_junho_sc",
    "vl_rem_julho_sc",
    "vl_rem_agosto_sc",
    "vl_rem_setembro_sc",
    "vl_rem_outubro_sc",
    "vl_rem_novembro_sc"
]

# Transform columns to double type
for column in double_columns:
    rais = rais.withColumn(column, col(column).cast("double"))

# Add a new column "uf" by extracting the first two characters from the "municipio" column
rais = rais.withColumn("uf", col("municipio").cast("string").substr(1, 2).cast("int"))


# Print the updated schema
rais.printSchema()

root
 |-- bairros_sp: string (nullable = true)
 |-- bairros_fortaleza: string (nullable = true)
 |-- bairros_rj: string (nullable = true)
 |-- causa_afastamento_1: integer (nullable = true)
 |-- causa_afastamento_2: integer (nullable = true)
 |-- causa_afastamento_3: integer (nullable = true)
 |-- motivo_desligamento: integer (nullable = true)
 |-- cbo_ocupacao_2002: string (nullable = true)
 |-- cnae_2.0_classe: integer (nullable = true)
 |-- cnae_95_classe: integer (nullable = true)
 |-- distritos_sp: string (nullable = true)
 |-- vinculo_ativo_31/12: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)
 |-- faixa_hora_contrat: integer (nullable = true)
 |-- faixa_remun_dezem_(sm): integer (nullable = true)
 |-- faixa_remun_media_(sm): integer (nullable = true)
 |-- faixa_tempo_emprego: integer (nullable = true)
 |-- escolaridade_apos_2005: double (nullable = true)
 |-- qtd_hora_contr: double (nullable = true)
 |-- idade: double (nullable = true)
 |-- ind_cei_vincul

In [21]:
# Get row count
rows = rais.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Rows count : 65921194


In [ ]:
# Define output_path
output_path = "/content/drive/MyDrive/desafio_mba/rais/staging"

# Check if the directory exists and create it if necessary
if not os.path.exists(output_path):
    os.makedirs(output_path)

(
    rais
    .coalesce(50)
    .write.mode('overwrite')
    .partitionBy("uf")
    .format('parquet')
    .save('/content/drive/MyDrive/desafio_mba/rais/staging')
)

In [7]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read the Parquet files
staging = spark.read.parquet('/content/drive/MyDrive/mba/xpe/desafio01/staging')

# Perform operations on the data
staging.printSchema()

root
 |-- bairros_sp: string (nullable = true)
 |-- bairros_fortaleza: string (nullable = true)
 |-- bairros_rj: string (nullable = true)
 |-- causa_afastamento_1: string (nullable = true)
 |-- causa_afastamento_2: string (nullable = true)
 |-- causa_afastamento_3: string (nullable = true)
 |-- motivo_desligamento: string (nullable = true)
 |-- cbo_ocupacao_2002: string (nullable = true)
 |-- cnae_2.0_classe: string (nullable = true)
 |-- cnae_95_classe: string (nullable = true)
 |-- distritos_sp: string (nullable = true)
 |-- vinculo_ativo_31/12: string (nullable = true)
 |-- faixa_etaria: string (nullable = true)
 |-- faixa_hora_contrat: string (nullable = true)
 |-- faixa_remun_dezem_(sm): string (nullable = true)
 |-- faixa_remun_media_(sm): string (nullable = true)
 |-- faixa_tempo_emprego: string (nullable = true)
 |-- escolaridade_apos_2005: string (nullable = true)
 |-- qtd_hora_contr: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- ind_cei_vinculado: string 

In [ ]:
rows = staging.count()
print(f"DataFrame Rows count : {rows}")

# 1.0 Questionario

## Qual é o SEGUNDO motivo de desligamento mais frequente?

## Qual é o estado com a maior média de renda nominal do Brasil?

## Qual é a média da renda nominal da UF 31 (utilize a renda média nominal)?

## Quantas pessoas, no estado 21, possuem renda nominal média menor do que o salário-mínimo de 2020 (1039 reais)?

## Qual foi a média de horas trabalhadas na UF 16?

## Qual é a diferença entre os salários médios nominais entre categorias de sexo?

## Qual é a diferença entre os salários médios nominais entre categorias de sexo para os trabalhadores da área de tecnologia (CNAE 2.0 Classe = 62040)?